In [1]:
# main directory
%cd /content/drive/MyDrive/Colab Notebooks/regaetton_songs_nlp
# get songs directory
%cd get_songs

/content/drive/MyDrive/Colab Notebooks/regaetton_songs_nlp
/content/drive/MyDrive/Colab Notebooks/regaetton_songs_nlp/get_songs


In [2]:
import pandas as pd

In [3]:
lyrics_dict = pd.read_pickle('lyrics.pickle')
df = pd.DataFrame.from_dict(lyrics_dict)
df.head()

,/bad-bunny/,/maluma/,/sebastian-yatra/,/ozuna/,/camilo-echeverry/,/karol-g/,/la-reina-del-flow/,/daddy-yankee/,/j-balvin/,/sech/,/don-omar/,/farruko/,/arcangel/,/danny-ocean/,/myke-towers/,/anuel/,/nicky-jam/,/natti-natasha/,/jay-wheeler/,/paloma-mami/,/yandel/,/latin-dreams/,/wisin-yandel/,/nio-garcia/,/vico-c/,/c-tangana/,/dalex/,/justin-quiles/,/nigga/,/wisin/,/funky/,/lil-silvio-e-el-vega/,/la-factoria/,/lunay/,/zion-y-lennox/,/tito-el-bambino/,/rakim-ken-y/,/bryant-myers/,/tony-dize/,/chino-nacho/,...,/michael-stuart/,/leslie-shaw/,/rk/,/nore/,/opalo/,/randy/,/tony-tun-tun/,/cruzito/,/keenv/,/lorena-santos-es/,/el-monje/,/bachatas-ballenatos/,/ghetto-kids/,/reggaeton-nios/,/dylan-fuentes/,/million-stylez/,/el-medico/,/chapa-c/,/alexis-fido/,/nina-sky/,/fernanda-mx/,/tres-dedos/,/leebrian/,/bandaga/,/jimmy-bad-boy/,/tainy/,/velocity-girl/,/bellakath/,/don-kikas/,/izaak/,/la-fabula/,/gallego/,/los-tigrillos/,/latino-way/,/juakin-saenz/,/agrupacion-52/,/joako-fabrega/,/eliel/,/wibal-alex/,/magnate/
/bad-bunny/dakiti-part-jhay-cortez/,"Baby, ya yo me enteré, se nota cuando me ve\nA...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
By,Compuesta por: Arcángel / Henry De La Prida ...,Compuesta por: Austín Santos / Bryan Snaider...,¿Sabes quién compuso esta canción? Envíanoslo.,¿Sabes quién compuso esta canción? Envíanoslo.,Compuesta por: Alejandro Sanz. ¿Los datos es...,¿Sabes quién compuso esta canción? Envíanoslo.,Compuesta por: Brian. ¿Los datos están equiv...,¿Sabes quién compuso esta canción? Envíanoslo.,¿Sabes quién compuso esta canción? Envíanoslo.,¿Sabes quién compuso esta canción? Envíanoslo.,¿Sabes quién compuso esta canción? Envíanoslo.,¿Sabes quién compuso esta canción? Envíanoslo.,¿Sabes quién compuso esta canción? Envíanoslo.,¿Sabes quién compuso esta canción? Envíanoslo.,¿Sabes quién compuso esta canción? Envíanoslo.,Compuesta por: Anuel AA / Cardi B / Camille...,¿Sabes quién compuso esta canción? Envíanoslo.,¿Sabes quién compuso esta canción? Envíanoslo.,Compuesta por: Jay Wheeler. ¿Los datos están...,Compuesta por: Alizzz / C. Tangana / Paloma ...,Compuesta por: Dynell / Tainy / Yandel. ¿Los...,¿Sabes quién compuso esta canción? Envíanoslo.,¿Sabes quién compuso esta canción? Envíanoslo.,¿Sabes quién compuso esta canción? Envíanoslo.,¿Sabes quién compuso esta canción? Envíanoslo.,¿Sabes quién compuso esta canción? Envíanoslo.,¿Sabes quién compuso esta canción? Envíanoslo.,Compuesta por: Hector David Lamboy / Joshua ...,¿Sabes quién compuso esta canción? Envíanoslo.,¿Sabes quién compuso esta canción? Envíanoslo.,Compuesta por: Luis Marrero. ¿Los datos está...,¿Sabes quién compuso esta canción? Envíanoslo.,¿Sabes quién compuso esta canción? Envíanoslo.,Compuesta por: Chris Jeday / Eduardo Vargas ...,¿Sabes quién compuso esta canción? Envíanoslo.,¿Sabes quién compuso esta canción? Envíanoslo.,Compuesta por: Ken-Y / Rakim. ¿Los datos est...,¿Sabes quién compuso esta canción? Envíanoslo.,¿Sabes quién compuso esta canción? Envíanoslo.,¿Sabes quién compuso esta canción? Envíanoslo.,...,¿Sabes quién compuso esta canción? Envíanoslo.,¿Sabes quién compuso esta canción? Envíanoslo.,¿Sabes quién compuso esta canción? Envíanoslo.,¿Sabes quién compuso esta canción? Envíanoslo.,¿Sabes quién compuso esta canción? Envíanoslo.,¿Sabes quién compuso esta canción? Envíanoslo.,¿Sabes quién compuso esta canción? Envíanoslo.,¿Sabes quién compuso esta canción? Envíanoslo.,¿Sabes quién compuso esta canción? Envíanoslo.,¿Sabes quién compuso esta canción? Envíanoslo.,Compuesta por: El Monje. ¿Los datos están eq...,¿Sabes quién compuso esta canción? Envíanoslo.,¿Sabes quién compuso esta canción? Envíanoslo.,¿Sabes quién compuso esta canción? Envíanoslo.,¿Sabes quién compu

# Understanding what we want

Right now, we have in the columns each artist name and the indexes are the song names. What we want as our dataframe is a numerical index. Then, we will want the following columns: song_name, artist, lyrics.

The By column contains the information of who wrote the songs but for now we are not going to use it.

# Getting the numerical index

In [4]:
df.reset_index(inplace=True)
df.head()

,index,/bad-bunny/,/maluma/,/sebastian-yatra/,/ozuna/,/camilo-echeverry/,/karol-g/,/la-reina-del-flow/,/daddy-yankee/,/j-balvin/,/sech/,/don-omar/,/farruko/,/arcangel/,/danny-ocean/,/myke-towers/,/anuel/,/nicky-jam/,/natti-natasha/,/jay-wheeler/,/paloma-mami/,/yandel/,/latin-dreams/,/wisin-yandel/,/nio-garcia/,/vico-c/,/c-tangana/,/dalex/,/justin-quiles/,/nigga/,/wisin/,/funky/,/lil-silvio-e-el-vega/,/la-factoria/,/lunay/,/zion-y-lennox/,/tito-el-bambino/,/rakim-ken-y/,/bryant-myers/,/tony-dize/,...,/michael-stuart/,/leslie-shaw/,/rk/,/nore/,/opalo/,/randy/,/tony-tun-tun/,/cruzito/,/keenv/,/lorena-santos-es/,/el-monje/,/bachatas-ballenatos/,/ghetto-kids/,/reggaeton-nios/,/dylan-fuentes/,/million-stylez/,/el-medico/,/chapa-c/,/alexis-fido/,/nina-sky/,/fernanda-mx/,/tres-dedos/,/leebrian/,/bandaga/,/jimmy-bad-boy/,/tainy/,/velocity-girl/,/bellakath/,/don-kikas/,/izaak/,/la-fabula/,/gallego/,/los-tigrillos/,/latino-way/,/juakin-saenz/,/agrupacion-52/,/joako-fabrega/,/eliel/,/wibal-alex/,/magnate/
0,/bad-bunny/dakiti-part-jhay-cortez/,"Baby, ya yo me enteré, se nota cuando me ve\nA...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,By,Compuesta por: Arcángel / Henry De La Prida ...,Compuesta por: Austín Santos / Bryan Snaider...,¿Sabes quién compuso esta canción? Envíanoslo.,¿Sabes quién compuso esta canción? Envíanoslo.,Compuesta por: Alejandro Sanz. ¿Los datos es...,¿Sabes quién compuso esta canción? Envíanoslo.,Compuesta por: Brian. ¿Los datos están equiv...,¿Sabes quién compuso esta canción? Envíanoslo.,¿Sabes quién compuso esta canción? Envíanoslo.,¿Sabes quién compuso esta canción? Envíanoslo.,¿Sabes quién compuso esta canción? Envíanoslo.,¿Sabes quién compuso esta canción? Envíanoslo.,¿Sabes quién compuso esta canción? Envíanoslo.,¿Sabes quién compuso esta canción? Envíanoslo.,¿Sabes quién compuso esta canción? Envíanoslo.,Compuesta por: Anuel AA / Cardi B / Camille...,¿Sabes quién compuso esta canción? Envíanoslo.,¿Sabes quién compuso esta canción? Envíanoslo.,Compuesta por: Jay Wheeler. ¿Los datos están...,Compuesta por: Alizzz / C. Tangana / Paloma ...,Compuesta por: Dynell / Tainy / Yandel. ¿Los...,¿Sabes quién compuso esta canción? Envíanoslo.,¿Sabes quién compuso esta canción? Envíanoslo.,¿Sabes quién compuso esta canción? Envíanoslo.,¿Sabes quién compuso esta canción? Envíanoslo.,¿Sabes quién compuso esta canción? Envíanoslo.,¿Sabes quién compuso esta canción? Envíanoslo.,Compuesta por: Hector David Lamboy / Joshua ...,¿Sabes quién compuso esta canción? Envíanoslo.,¿Sabes quién compuso esta canción? Envíanoslo.,Compuesta por: Luis Marrero. ¿Los datos está...,¿Sabes quién compuso esta canción? Envíanoslo.,¿Sabes quién compuso esta canción? Envíanoslo.,Compuesta por: Chris Jeday / Eduardo Vargas ...,¿Sabes quién compuso esta canción? Envíanoslo.,¿Sabes quién compuso esta canción? Envíanoslo.,Compuesta por: Ken-Y / Rakim. ¿Los datos est...,¿Sabes quién compuso esta canción? Envíanoslo.,¿Sabes quién compuso esta canción? Envíanoslo.,...,¿Sabes quién compuso esta canción? Envíanoslo.,¿Sabes quién compuso esta canción? Envíanoslo.,¿Sabes quién compuso esta canción? Envíanoslo.,¿Sabes quién compuso esta canción? Envíanoslo.,¿Sabes quién compuso esta canción? Envíanoslo.,¿Sabes quién compuso esta canción? Envíanoslo.,¿Sabes quién compuso esta canción? Envíanoslo.,¿Sabes quién compuso esta canción? Envíanoslo.,¿Sabes quién compuso esta canción? Envíanoslo.,¿Sabes quién compuso esta canción? Envíanoslo.,Compuesta por: El Monje. ¿Los datos están eq...,¿Sabes quién compuso esta canción? Envíanoslo.,¿Sabes quién compuso esta canción? Envíanoslo.,¿Sabes quién compuso esta canción? Envíanoslo.,¿Sabes quién compuso esta canción? Envíanoslo.,¿Sabes quién compuso esta 

# Getting the artist_name column


In [5]:
df_artist_name = pd.melt(df, id_vars=['index'])
df_artist_name.head()

,index,variable,value
0,/bad-bunny/dakiti-part-jhay-cortez/,/bad-bunny/,"Baby, ya yo me enteré, se nota cuando me ve\nA..."
1,By,/bad-bunny/,Compuesta por: Arcángel / Henry De La Prida ...
2,/bad-bunny/la-ultima-vez/,/bad-bunny/,[Anuel AA]\nYo te perdí la última vez que te v...
3,/bad-bunny/como-se-siente-remix-part-jhay-cortez/,/bad-bunny/,"(Yeh-yeh-yeh-yeh, ey)\n\nNos matamo' (nos mata..."
4,/bad-bunny/si-estuviesemos-juntos/,/bad-bunny/,"Yeh, yeh, yeh, yeh, yeh\nYeh, yeh, yeh, yeh\nY..."


In [10]:
df_dropna = df_artist_name.dropna().reset_index(drop=True)

# Checking for duplicates and na values
print(df_dropna.isnull().sum())
print(f'Duplicate rows {df_dropna.duplicated().sum()}')

index       0
variable    0
value       0
dtype: int64
Duplicate rows 0


In [11]:
df_dropna.head(5)

,index,variable,value
0,/bad-bunny/dakiti-part-jhay-cortez/,/bad-bunny/,"Baby, ya yo me enteré, se nota cuando me ve\nA..."
1,By,/bad-bunny/,Compuesta por: Arcángel / Henry De La Prida ...
2,/bad-bunny/la-ultima-vez/,/bad-bunny/,[Anuel AA]\nYo te perdí la última vez que te v...
3,/bad-bunny/como-se-siente-remix-part-jhay-cortez/,/bad-bunny/,"(Yeh-yeh-yeh-yeh, ey)\n\nNos matamo' (nos mata..."
4,/bad-bunny/si-estuviesemos-juntos/,/bad-bunny/,"Yeh, yeh, yeh, yeh, yeh\nYeh, yeh, yeh, yeh\nY..."


In [15]:
print(df_dropna['index'].value_counts(ascending=False).head())
df_no_by = df_dropna[df_dropna['index'] != 'By'].reset_index(drop=True)
df_no_by['index'].value_counts(ascending=False).head()

By                               200
/arcangel/917185/                  1
/alexis-y-fido/1488710/            1
/randy/313674/                     1
/tito-el-bambino/alzo-mi-voz/      1
Name: index, dtype: int64


/arcangel/917185/             1
/randy/313674/                1
/don-omar/286936/             1
/casper-magico/fantasmita/    1
/la-factoria/1690446/         1
Name: index, dtype: int64

In [16]:
columns = ['song_name', 'artist', 'lyrics']
clean_df = df_no_by.copy()
clean_df.columns = columns
clean_df.head()

,song_name,artist,lyrics
0,/bad-bunny/dakiti-part-jhay-cortez/,/bad-bunny/,"Baby, ya yo me enteré, se nota cuando me ve\nA..."
1,/bad-bunny/la-ultima-vez/,/bad-bunny/,[Anuel AA]\nYo te perdí la última vez que te v...
2,/bad-bunny/como-se-siente-remix-part-jhay-cortez/,/bad-bunny/,"(Yeh-yeh-yeh-yeh, ey)\n\nNos matamo' (nos mata..."
3,/bad-bunny/si-estuviesemos-juntos/,/bad-bunny/,"Yeh, yeh, yeh, yeh, yeh\nYeh, yeh, yeh, yeh\nY..."
4,/bad-bunny/te-bote/,/bad-bunny/,"[Darell]\nEste es el verdadero remix, baby\n\n..."


# Clean the values for song_name and artist columns

Now, what we want is to clean the values in the song_name and artist columns. Because the data was gather from a webpage it looks a little bit messy. For example, we want the value Bad Bunny instead of /bad-bunny/ and the same for the song names.

In [17]:
def get_artist_name(artist):
    artist = artist.replace('/', '')
    artist = artist.replace('-', ' ')
    artist = artist.title()

    return artist

clean_df.artist = clean_df.artist.apply(get_artist_name)
print(clean_df.artist.head())
print(clean_df.artist.tail())

0    Bad Bunny
1    Bad Bunny
2    Bad Bunny
3    Bad Bunny
4    Bad Bunny
Name: artist, dtype: object
9471    Magnate
9472    Magnate
9473    Magnate
9474    Magnate
9475    Magnate
Name: artist, dtype: object


In [18]:
clean_df.tail()

,song_name,artist,lyrics
9471,/magnate/1238536/,Magnate,Fue como la primera vez\nMe acuerdo como suced...
9472,/magnate/1246010/,Magnate,Ahi algo que me tiene confundido\nY loco en es...
9473,/magnate/1246007/,Magnate,No aguanto un dia mas\nTal vez yo fingi\n\nNo ...
9474,/magnate/1246009/,Magnate,Magnate\nMagnate\nPor mi\n\nEmpecemos el juego...
9475,/magnate/1238537/,Magnate,dime porque estoy aqui\nextrañandote a ti\nsi ...


# Song name column

With this column we have a problem. When we scrapped the songs, for some of them we did not get the name, instead we got the artist name and a number. So, the idea is to leave just the song name and for the ones with the number we will replace them with a None, this because in the future it could be a funny project to try to predict the song name using some ML.

In [20]:
clean_df.song_name.head()

0                  /bad-bunny/dakiti-part-jhay-cortez/
1                            /bad-bunny/la-ultima-vez/
2    /bad-bunny/como-se-siente-remix-part-jhay-cortez/
3                   /bad-bunny/si-estuviesemos-juntos/
4                                  /bad-bunny/te-bote/
Name: song_name, dtype: object

In [28]:
def get_song_name(song):

    song = song.replace('/', ' ').strip()
    song = song.split()[-1]
    if song.isnumeric():
        return None
    else:
        song = song.replace('-', ' ')
        return song.capitalize()

clean_df.song_name = clean_df.song_name.apply(get_song_name)

In [32]:
print(f'Number of songs without a title: {clean_df.song_name.isnull().sum()}')
clean_df.head(10)

Number of songs without a title: 5653


,song_name,artist,lyrics
0,Dakiti part jhay cortez,Bad Bunny,"Baby, ya yo me enteré, se nota cuando me ve\nA..."
1,La ultima vez,Bad Bunny,[Anuel AA]\nYo te perdí la última vez que te v...
2,Como se siente remix part jhay cortez,Bad Bunny,"(Yeh-yeh-yeh-yeh, ey)\n\nNos matamo' (nos mata..."
3,Si estuviesemos juntos,Bad Bunny,"Yeh, yeh, yeh, yeh, yeh\nYeh, yeh, yeh, yeh\nY..."
4,Te bote,Bad Bunny,"[Darell]\nEste es el verdadero remix, baby\n\n..."
5,6 rings,Bad Bunny,"Yeh, ey\n\nA veces, me pregunto si hago bien e..."
6,Amorfoda,Bad Bunny,No quiero que más nadie me hable de amor\nYa m...
7,Una vez part mora,Bad Bunny,"Lo nuestro no te conviene, eh\nPero él ya no t..."
8,Soy peor remix part j balvin ozuna y arcangel,Bad Bunny,"No puedo confiar (no, no)\nEn nadie confió (no..."
9,La santa part daddy yankee,Bad Bunny,"Tú no ere' una santa, ni yo soy un santo\nNos ..."
